# Training winter forecasts

Author: Martin Wegmann

Email: martin.wegmann@unibe.ch

The goal of this notebook is to train the monthly prediction of the DJF seasons T2M and SLP from November SSTs. 
The files that are read in are already normalized. Check the preprocessing notebook for that. 
We try out a couple of different architectures, including simple linear regression.

Note that this notebook is very non-automaized. Time was not a big constraint in my work.

This is an example how to do it for the winter season. The exact same structure is done for the summer season, meaning May SSTs as input for JJA T2M and SLP predictions.

# load libraries

In [1]:
from sklearn.utils import shuffle
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
import seaborn as sns
import pickle
import os
from keras import layers
from keras import models
from tensorflow.keras import models
from collections import OrderedDict
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.layers import Dense, SimpleRNN, LSTM, GRU, Reshape
from keras import initializers
from keras.models import Sequential
from keras.optimizers import RMSprop
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import datasets, linear_model
from sklearn.utils import shuffle

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

from tensorflow.keras import Model, Input, regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Add, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import image
from sklearn.model_selection import train_test_split 

from sklearn.svm import SVR
from sklearn import svm

2024-09-19 17:15:05.459684: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from lazypredict.Supervised import LazyRegressor

from sklearn.utils import shuffle

In [3]:
resize_method = tf.image.ResizeMethod.NEAREST_NEIGHBOR

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
#

Num GPUs Available:  0


### define functions

In [ ]:
def write_check_txt(text="check"):
    with open(text+'.txt', 'w') as f:
        f.write(text)
    return print(text)


In [ ]:
def val_loss_plot(history,folder,name="example"):
    fig = plt.figure(figsize=(8, 4), dpi= 200)
    loss =history.history["loss"]
    val_loss =history.history["val_loss"]
    epochs=range(1,len(loss)+1)

    print(np.min(val_loss))


    print(np.min(loss))

    plt.figure()

    plt.plot(epochs,loss,"b",color="blue",label="Training loss")
    plt.plot(epochs,val_loss,"b",color="red",label="Validation loss")
    plt.title("Loss Curves for "+name)
    plt.title("VL: "+str(round(np.min(val_loss),2)),loc="left")
    plt.title("TL: "+str(round(np.min(loss),2)),loc="right")
    plt.legend()
    
    plt.savefig(folder+"Loss Curves for "+name+".png")
    plt.show()
    return print("saved in "+folder+"Loss Curves for "+name+".png")

### define target season and variable

we generate 5 members for each target

In [ ]:
target="t2m_DJF"# target_list=["t2m_dec","t2m_jan","t2m_feb","t2m_DJF","slp_DJF"]

In [ ]:
stat_model_member="01"

### define folders

In [ ]:
save_folder="/storage/homefs/mawegmann/data/"

In [ ]:
readin_mpige="/storage/homefs/mawegmann/data/mpi_ge/"

In [ ]:
readin_data="/storage/homefs/mawegmann/data/mpi_ge/"

# read in normalized, masked SSTs

What kind of data sets do we have in here?

* MPI-GE Control

* MPI-GE Hist

* MPI-GE RCP26

* CODA SSTs 1850-1899 for training

* CODA SSTs 1900-2015 for testing

We focus on November mean SSTs here.

All training datasets are normalized with the same mean and standard deviation (coming from Mpi-GE).
The test data is normalized by the mean and standard deviation of the test dataset.

### mpi ge ssts

In [ ]:
thetao_nov = xr.open_dataset(readin_mpige+"thetao7_OImon_traintest_nov_lowlow_supermasked_anomnorm.nc")
thetao_nov = thetao_nov.thetao.drop('lev')
thetao_nov= thetao_nov.squeeze('lev')

thetao_nov_valid = xr.open_dataset(readin_mpige+"thetao7_OImon_validate_nov_lowlow_supermasked_anomnorm.nc")
thetao_nov_valid = thetao_nov_valid.thetao.drop('lev')
thetao_nov_valid= thetao_nov_valid.squeeze('lev')


In [ ]:
thetao_nov_ctl = xr.open_dataset(readin_mpige+"thetao_control_nov_lowlow_supermasked_anomnorm.nc")
thetao_nov_ctl = thetao_nov_ctl.thetao.drop('lev')
thetao_nov_ctl= thetao_nov_ctl.squeeze('lev')


In [ ]:
thetao_nov_rcp26 = xr.open_dataset(readin_mpige+"thetao7_OImon_traintest_nov_lowlow_supermasked_rcp26_anomnorm.nc")
thetao_nov_rcp26 = thetao_nov_rcp26.thetao.drop('lev')
thetao_nov_rcp26= thetao_nov_rcp26.squeeze('lev')


### coda ssts

In [ ]:
thetao_nov_codaearly = xr.open_dataset(readin_data+"sst.mon.mean_18501899_nov_lowlow_supermasked_anomnorm.nc")
thetao_nov_codaearly = thetao_nov_codaearly.sst


In [ ]:
thetao_nov_codalate = xr.open_dataset(readin_data+"sst.mon.mean_19002014_nov_lowlow_supermasked_anomnorm.nc")
thetao_nov_codalate = thetao_nov_codalate.sst

thetao_nov_codalate_refwindow=thetao_nov_codalate[51:81,:,:]
thetao_nov_codalate_climate=thetao_nov_codalate_refwindow.mean("time")

# Read in normalized Atmospheric Fields

What kind of data sets do we have in here?

* MPI-GE Control

* MPI-GE Hist

* MPI-GE RCP26

* 20CRv3 ensmean 1850-1899 for training

* 20Crv3 ensmean 1900-2015 for testing

We focus on December, January and February fields here as well as the DJF mean. 

We just look at 2m temperature and Sea Level Pressure.

All training datasets are normalized with the same mean and standard deviation (coming from Mpi-GE).
The test data is normalized by the mean and standard deviation of the test dataset.

For some reason the SST data has only 99 members in MPI-GE instead of 100, so we have to cut our atmospheric data a bit

### mpi ge

In [ ]:
t2m_DJF = xr.open_dataset(readin_mpige+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_anomnorm.nc")
t2m_DJF =t2m_DJF.tas
t2m_DJF = t2m_DJF[:15190,:,:]

lons=t2m_DJF.lon
lats=t2m_DJF.lat


In [ ]:
t2m_dec= xr.open_dataset(readin_mpige+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_dec_lowlow_anomnorm.nc")
t2m_dec =t2m_dec.tas
t2m_dec = t2m_dec[:15190,:,:]


In [ ]:
t2m_jan = xr.open_dataset(readin_mpige+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_jan_lowlow_anomnorm.nc")
t2m_jan =t2m_jan.tas
t2m_jan = t2m_jan[:15190,:,:]


In [ ]:
t2m_feb = xr.open_dataset(readin_mpige+"tas_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_feb_lowlow_anomnorm.nc")
t2m_feb =t2m_feb.tas
t2m_feb = t2m_feb[:15190,:,:]


In [ ]:
slp_DJF = xr.open_dataset(readin_mpige+"psl_Amon_MPI-ESM_historical_all_i1850p3_185001-200512_traintest_DJF_lowlow_anomnorm.nc")
slp_DJF =slp_DJF.psl
slp_DJF = slp_DJF[:15190,:,:]


In [ ]:
t2m_DJF_ctl = xr.open_dataset(readin_mpige+"tas_Amon_mpige_control_DJF_lowlow_anomnorm.nc")
t2m_DJF_ctl =t2m_DJF_ctl.tas



In [ ]:
t2m_dec_ctl = xr.open_dataset(readin_mpige+"tas_Amon_mpige_control_dec_lowlow_anomnorm.nc")
t2m_dec_ctl =t2m_dec_ctl.tas



In [ ]:
t2m_jan_ctl = xr.open_dataset(readin_mpige+"tas_Amon_mpige_control_jan_lowlow_anomnorm.nc")
t2m_jan_ctl =t2m_jan_ctl.tas


In [ ]:
t2m_feb_ctl = xr.open_dataset(readin_mpige+"tas_Amon_mpige_control_feb_lowlow_anomnorm.nc")
t2m_feb_ctl =t2m_feb_ctl.tas


In [ ]:
slp_DJF_ctl = xr.open_dataset(readin_mpige+"psl_Amon_mpige_control_DJF_lowlow_anomnorm.nc")
slp_DJF_ctl =slp_DJF_ctl.psl


In [ ]:
t2m_DJF_rcp26 = xr.open_dataset(readin_mpige+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_DJF_lowlow_anomnorm.nc")
t2m_DJF_rcp26  =t2m_DJF_rcp26.tas

t2m_DJF_rcp26= t2m_DJF_rcp26[:9114,:,:]


In [ ]:
t2m_dec_rcp26 = xr.open_dataset(readin_mpige+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_dec_lowlow_anomnorm.nc")
t2m_dec_rcp26  =t2m_dec_rcp26.tas

t2m_dec_rcp26= t2m_dec_rcp26[:9114,:,:]


In [ ]:
t2m_jan_rcp26 = xr.open_dataset(readin_mpige+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_jan_lowlow_anomnorm.nc")
t2m_jan_rcp26  =t2m_jan_rcp26.tas
t2m_jan_rcp26= t2m_jan_rcp26[:9114,:,:]


In [ ]:
t2m_feb_rcp26 = xr.open_dataset(readin_mpige+"tas_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_feb_lowlow_anomnorm.nc")
t2m_feb_rcp26  =t2m_feb_rcp26.tas
t2m_feb_rcp26= t2m_feb_rcp26[:9114,:,:]


In [ ]:
slp_DJF_rcp26 = xr.open_dataset(readin_mpige+"psl_Amon_MPI-ESM_rcp26_all_i2005p3_200601-209912_traintest_DJF_lowlow_anomnorm.nc")
slp_DJF_rcp26  =slp_DJF_rcp26.psl
slp_DJF_rcp26= slp_DJF_rcp26[:9114,:,:]


### 20CRv3

In [ ]:
t2m_DJF_codaearly = xr.open_dataset(readin_data+"air.2m.mon.mean_DJF_18511900_lowlow_anomnorm.nc")
t2m_DJF_codaearly  =t2m_DJF_codaearly.air


In [ ]:
slp_DJF_codaearly = xr.open_dataset(readin_data+"prmsl.mon.mean_DJF_18511900_lowlow_anomnorm.nc")
slp_DJF_codaearly  =slp_DJF_codaearly.prmsl


In [ ]:
t2m_dec_codaearly = xr.open_dataset(readin_data+"air.2m.mon.mean_18501899_dec_lowlow_anomnorm.nc")
t2m_dec_codaearly  =t2m_dec_codaearly.air


In [ ]:
t2m_jan_codaearly = xr.open_dataset(readin_data+"air.2m.mon.mean_18511900_jan_lowlow_anomnorm.nc")
t2m_jan_codaearly  =t2m_jan_codaearly.air


In [ ]:
t2m_feb_codaearly = xr.open_dataset(readin_data+"air.2m.mon.mean_18511900_feb_lowlow_anomnorm.nc")
t2m_feb_codaearly  =t2m_feb_codaearly.air


In [ ]:
t2m_DJF_codalate = xr.open_dataset(readin_data+"air.2m.mon.mean_DJF_19012015_lowlow_anomnorm.nc")
t2m_DJF_codalate  =t2m_DJF_codalate.air


In [ ]:
slp_DJF_codalate = xr.open_dataset(readin_data+"prmsl.mon.mean_DJF_19012015_lowlow_anomnorm.nc")
slp_DJF_codalate  =slp_DJF_codalate.prmsl


In [ ]:
t2m_dec_codalate = xr.open_dataset(readin_data+"air.2m.mon.mean_19002014_dec_lowlow_anomnorm.nc")
t2m_dec_codalate  =t2m_dec_codalate.air


In [ ]:
t2m_jan_codalate = xr.open_dataset(readin_data+"air.2m.mon.mean_19012015_jan_lowlow_anomnorm.nc")
t2m_jan_codalate  =t2m_jan_codalate.air


In [ ]:
t2m_feb_codalate = xr.open_dataset(readin_data+"air.2m.mon.mean_19012015_feb_lowlow_anomnorm.nc")
t2m_feb_codalate  =t2m_feb_codalate.air


# prepare fields for training

here we can stack the data to multiple channels or postprocess in any form we need to

In [ ]:
sst_hist_nov=thetao_nov.reset_index("time",drop=True)


In [ ]:
sst_ctl_nov=thetao_nov_ctl.reset_index("time",drop=True)


In [ ]:
sst_rcp26_nov=thetao_nov_rcp26.reset_index("time",drop=True)


In [ ]:
sst_hist_valid_nov=thetao_nov_valid.reset_index("time",drop=True)


In [ ]:
sst_coda_20th_nov=thetao_nov_codalate.reset_index("time",drop=True)


In [ ]:
sst_coda_19th_nov=thetao_nov_codaearly.reset_index("time",drop=True)


Create empty containers to stack (not needed here).

That way you can stack multiple channels (such as sep, oct, nov ssts)

In [ ]:
sst_stacked_hist = []
sst_stacked_ctl = []
sst_stacked_rcp26 = []
sst_stacked_coda_20th= []
sst_stacked_coda_19th= []


In [ ]:
#data_mpi.append(ds_mpi)
#data_mpi.append(dst_mpi)
sst_stacked_hist.append(sst_hist_nov)



In [ ]:
#data_ctl.append(ds_ctl)
#data_ctl.append(dst_ctl)
sst_stacked_ctl.append(sst_ctl_nov)



In [ ]:
#data_rcp26.append(ds_rcp26)
#data_rcp26.append(dst_rcp26)
sst_stacked_rcp26.append(sst_rcp26_nov)



In [ ]:
#data_coda.append(ds_test_coda)
#data_coda.append(dst_test_coda)
sst_stacked_coda_20th.append(sst_coda_20th_nov)



In [ ]:
#data_codaearly.append(ds_codaearly)
#data_codaearly.append(dst_codaearly)
sst_stacked_coda_19th.append(sst_coda_19th_nov)



In [ ]:
sst_stacked_hist = xr.concat(sst_stacked_hist, 'level').transpose('time', 'lat', 'lon', 'level')
sst_stacked_ctl = xr.concat(sst_stacked_ctl, 'level').transpose('time', 'lat', 'lon', 'level')
sst_stacked_rcp26 = xr.concat(sst_stacked_rcp26, 'level').transpose('time', 'lat', 'lon', 'level')
sst_stacked_coda_20th = xr.concat(sst_stacked_coda_20th, 'level').transpose('time', 'lat', 'lon', 'level')
sst_stacked_coda_19th= xr.concat(sst_stacked_coda_19th, 'level').transpose('time', 'lat', 'lon', 'level')

In [ ]:

if target=="t2m_DJF":
    target_hist=t2m_DJF
    target_coda_20th=t2m_DJF_codalate

    target_ctl=t2m_DJF_ctl
    target_rcp26=t2m_DJF_rcp26
    target_coda_19th=t2m_DJF_codaearly

if target=="t2m_dec":
    target_hist=t2m_dec
    target_coda_20th=t2m_dec_codalate

    target_ctl=t2m_dec_ctl
    target_rcp26=t2m_dec_rcp26
    target_coda_19th=t2m_dec_codaearly


if target=="t2m_jan":
    target_hist=t2m_jan
    target_coda_20th=t2m_jan_codalate

    target_ctl=t2m_jan_ctl
    target_rcp26=t2m_jan_rcp26
    target_coda_19th=t2m_jan_codaearly

if target=="t2m_feb":
    target_hist=t2m_feb
    target_coda_20th=t2m_feb_codalate

    target_ctl=t2m_feb_ctl
    target_rcp26=t2m_feb_rcp26
    target_coda_19th=t2m_feb_codaearly


if target=="slp_DJF":
    target_hist=slp_DJF
    target_coda_20th=slp_DJF_codalate

    target_ctl=slp_DJF_ctl
    target_rcp26=slp_DJF_rcp26
    target_coda_19th=slp_DJF_codaearly



### Convert NA fields into zeroes

In [ ]:
sst_stacked_hist=sst_stacked_hist.fillna(0).values
sst_stacked_coda_20th=sst_stacked_coda_20th.fillna(0).values


sst_stacked_coda_19th=sst_stacked_coda_19th.fillna(0).values
sst_stacked_ctl=sst_stacked_ctl.fillna(0).values
sst_stacked_rcp26=sst_stacked_rcp26.fillna(0).values


target_hist=target_hist.fillna(0).values
target_coda_20th=target_coda_20th.fillna(0).values
target_coda_19th=target_coda_19th.fillna(0).values
target_ctl=target_ctl.fillna(0).values
target_rcp26=target_rcp26.fillna(0).values


### define X values

In [ ]:

X_hist_ctl_rcp26=np.concatenate((sst_stacked_hist,sst_stacked_ctl,sst_stacked_rcp26),axis=0)
X_hist_coda_19th_ctl_rcp26=np.concatenate((sst_stacked_hist,sst_stacked_coda_19th,sst_stacked_ctl,sst_stacked_rcp26),axis=0)

### define Y values

In [ ]:
target_hist.shape

In [ ]:
target_coda_19th.shape

In [ ]:

y_hist_coda_19th_ctl_rcp26=np.concatenate((target_hist,target_coda_19th,target_ctl,target_rcp26),axis=0)

In [ ]:
target_hist.shape

In [ ]:
sst_stacked_hist.shape

In [ ]:
sst_stacked_coda_20th.shape

### shuffle data

In [ ]:
X_hist_coda_19th_ctl_rcp26_shuffled,y_hist_coda_19th_ctl_rcp26_shuffled = shuffle(X_hist_coda_19th_ctl_rcp26,y_hist_coda_19th_ctl_rcp26)

In [ ]:
X_hist_coda_19th_ctl_rcp26_shuffled.shape

In [ ]:
y_hist_coda_19th_ctl_rcp26_shuffled.shape

In [ ]:
nchannels=X_hist_coda_19th_ctl_rcp26_shuffled.shape[3]
N_train=X_hist_coda_19th_ctl_rcp26_shuffled.shape[0]
N_test=sst_stacked_coda_20th.shape[0]
nlat=X_hist_coda_19th_ctl_rcp26_shuffled.shape[1]
nlon=X_hist_coda_19th_ctl_rcp26_shuffled.shape[2]

### reshape data for regression

In [ ]:
X_hist_coda_19th_ctl_rcp26_shuffled4reg=X_hist_coda_19th_ctl_rcp26_shuffled.reshape((N_train,nlat*nlon))
y_hist_coda_19th_ctl_rcp26_shuffled4reg=y_hist_coda_19th_ctl_rcp26_shuffled.reshape((N_train,nlat*nlon))
sst_stacked_coda_20th4reg=sst_stacked_coda_20th.reshape((N_test,nlat*nlon))

### define regression model

In [ ]:
X_hist_coda_19th_ctl_rcp26_shuffled4reg_train=X_hist_coda_19th_ctl_rcp26_shuffled4reg[:round(X_hist_coda_19th_ctl_rcp26_shuffled4reg.shape[0]*0.8),:]
y_hist_coda_19th_ctl_rcp26_shuffled4reg_train=y_hist_coda_19th_ctl_rcp26_shuffled4reg[:round(y_hist_coda_19th_ctl_rcp26_shuffled4reg.shape[0]*0.8),:]
X_hist_coda_19th_ctl_rcp26_shuffled4reg_valid=X_hist_coda_19th_ctl_rcp26_shuffled4reg[round(X_hist_coda_19th_ctl_rcp26_shuffled4reg.shape[0]*0.8):,:]
y_hist_coda_19th_ctl_rcp26_shuffled4reg_valid=y_hist_coda_19th_ctl_rcp26_shuffled4reg[round(y_hist_coda_19th_ctl_rcp26_shuffled4reg.shape[0]*0.8):,:]


In [ ]:
y_hist_coda_19th_ctl_rcp26_shuffled4reg_valid.shape

### run regression

In [ ]:
regr=linear_model.LinearRegression(n_jobs=1)
trained_regr=regr.fit(X_hist_coda_19th_ctl_rcp26_shuffled4reg_train,y_hist_coda_19th_ctl_rcp26_shuffled4reg_train)

prediction_valid=trained_regr.predict(X_hist_coda_19th_ctl_rcp26_shuffled4reg_valid)
prediction_valid_2d=prediction_valid.reshape(X_hist_coda_19th_ctl_rcp26_shuffled4reg_valid.shape[0],nlat,nlon)

prediction=trained_regr.predict(sst_stacked_coda_20th4reg)
prediction_2d=prediction.reshape(N_test,nlat,nlon)

In [ ]:
y_valid_4corr=y_hist_coda_19th_ctl_rcp26_shuffled4reg_valid.reshape(X_hist_coda_19th_ctl_rcp26_shuffled4reg_valid.shape[0],nlat*nlon)
structure_dummy= np.arange(y_valid_4corr.shape[1], dtype=float)
structure_dummy.shape
for a in range(1,y_valid_4corr.shape[1]):
    one_R=np.corrcoef(prediction_valid[:,a], y_valid_4corr[:,a])
    structure_dummy[a]=one_R[0,1]
corr_matrix=structure_dummy.reshape(nlat,nlon)
corr_matrix.mean()

In [ ]:
y_test_4corr=target_coda_20th.reshape(N_test,nlat*nlon)
structure_dummy= np.arange(y_test_4corr.shape[1], dtype=float)
structure_dummy.shape
for a in range(1,y_test_4corr.shape[1]):
    one_R=np.corrcoef(prediction[:,a], y_test_4corr[:,a])
    structure_dummy[a]=one_R[0,1]
corr_matrix=structure_dummy.reshape(nlat,nlon)
corr_matrix.mean()

### define deep learning models

In [ ]:
class PeriodicPadding2D(tf.keras.layers.Layer):
    def __init__(self, pad_width, **kwargs):
        super().__init__(**kwargs)
        self.pad_width = pad_width

    def call(self, inputs, **kwargs):
        if self.pad_width == 0:
            return inputs
        inputs_padded = tf.concat(
            [inputs[:, :, -self.pad_width:, :], inputs, inputs[:, :, :self.pad_width, :]], axis=2)
        # Zero padding in the lat direction
        inputs_padded = tf.pad(inputs_padded, [[0, 0], [self.pad_width, self.pad_width], [0, 0], [0, 0]])
        return inputs_padded

    def get_config(self):
        config = super().get_config()
        config.update({'pad_width': self.pad_width})
        return config


class PeriodicConv2D(tf.keras.layers.Layer):
    def __init__(self, filters,
                 kernel_size,
                 conv_kwargs={},
                 **kwargs, ):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.conv_kwargs = conv_kwargs
        if type(kernel_size) is not int:
            assert kernel_size[0] == kernel_size[1], 'PeriodicConv2D only works for square kernels'
            kernel_size = kernel_size[0]
        pad_width = (kernel_size - 1) // 2
        self.padding = PeriodicPadding2D(pad_width)
        self.conv = Conv2D(
            filters, kernel_size, padding='valid', **conv_kwargs
        )

    def call(self, inputs):
        return self.conv(self.padding(inputs))

    def get_config(self):
        config = super().get_config()
        config.update({'filters': self.filters, 'kernel_size': self.kernel_size, 'conv_kwargs': self.conv_kwargs})
        return config

In [ ]:
def build_cnn(filters, kernels, input_shape, dr=0):
    """Fully convolutional network"""
    x = input = Input(shape=input_shape)
    for f, k in zip(filters[:-1], kernels[:-1]):
        x = PeriodicConv2D(f, k)(x)
        x = LeakyReLU()(x)
        if dr > 0: x = Dropout(dr)(x)
    output = PeriodicConv2D(filters[-1], kernels[-1])(x)
    return keras.models.Model(input, output)

In [ ]:
def build_cnn_pool(filters, kernels, input_shape, dr=0):
    """Fully convolutional network"""
    x = input = Input(shape=input_shape)
    for f, k in zip(filters[:-1], kernels[:-1]):
        x = PeriodicConv2D(f, k)(x)
        x = LeakyReLU()(x)
        x = MaxPooling2D((2, 2))(x)
        if dr > 0: x = Dropout(dr)(x)
    output = PeriodicConv2D(filters[-1], kernels[-1])(x)
    return keras.models.Model(input, output)

In [ ]:
def build_unet(input_shape, kernels,start_filters):
    x = input = Input(shape=input_shape)
    conv1 = PeriodicConv2D(filters=start_filters* 1, kernel_size=kernels)(x)
    conv1 = LeakyReLU()(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(0.6)(pool1)

    conv2 = PeriodicConv2D(filters=start_filters* 2, kernel_size=kernels)(pool1)
    conv2 = LeakyReLU()(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(0.6)(pool2)

    conv3 = PeriodicConv2D(filters=start_filters* 3, kernel_size=kernels)(pool2)
    conv3 = LeakyReLU()(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(0.6)(pool3)

    conv4 = PeriodicConv2D(filters=start_filters* 4, kernel_size=kernels)(pool3)
    conv4 = LeakyReLU()(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(0.6)(pool4)

    convm = Flatten()(pool4)
    convm = Dense(288,activation="relu")(convm)
    convm = Dropout(0.7)(convm)
    convm = Dense(3*6*64,activation="relu")(convm)
    convm = Dropout(0.7)(convm)


    t4 = tf.reshape(convm, shape=[-1, 3, 6, 64])
    deconv4 = tf.image.resize(t4,conv4.shape[1:3],method=resize_method)   
    uconv4 = concatenate([deconv4, pool3])
    uconv4 = PeriodicConv2D(filters=start_filters* 3, kernel_size=kernels)(uconv4)
    uconv4 = LeakyReLU()(uconv4)

    deconv3 = tf.image.resize(uconv4,conv3.shape[1:3],method=resize_method) 
    uconv3 = concatenate([deconv3, pool2])
    uconv3 = PeriodicConv2D(filters=start_filters * 2, kernel_size=kernels)(uconv3)
    uconv3 = LeakyReLU()(uconv3)
    
    deconv2 = tf.image.resize(uconv3,conv2.shape[1:3],method=resize_method)   
    uconv2 = concatenate([deconv2, pool1])
    uconv2 = PeriodicConv2D(filters=start_filters * 1, kernel_size=kernels)(uconv2)
    uconv2 = LeakyReLU()(uconv2)
    
    deconv1 = tf.image.resize(uconv2,conv1.shape[1:3],method=resize_method)   
    uconv1 = concatenate([deconv1, x])
    output_layer= PeriodicConv2D(filters=1, kernel_size=kernels)(uconv1)
    output = LeakyReLU()(output_layer)

    return keras.models.Model(input, output)
    

In [ ]:
def build_unet(input_shape, kernels,start_filters):
    x = input = Input(shape=input_shape)
    conv1 = PeriodicConv2D(filters=start_filters* 1, kernel_size=kernels)(x)
    conv1 = LeakyReLU()(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(0.6)(pool1)

    conv2 = PeriodicConv2D(filters=start_filters* 2, kernel_size=kernels)(pool1)
    conv2 = LeakyReLU()(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(0.6)(pool2)

    conv3 = PeriodicConv2D(filters=start_filters* 3, kernel_size=kernels)(pool2)
    conv3 = LeakyReLU()(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(0.6)(pool3)

    conv4 = PeriodicConv2D(filters=start_filters* 4, kernel_size=kernels)(pool3)
    conv4 = LeakyReLU()(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(0.6)(pool4)

    convm = Flatten()(pool4)
    convm = Dense(288,activation="relu")(convm)
    convm = Dropout(0.7)(convm)
    convm = Dense(3*6*64,activation="relu")(convm)
    convm = Dropout(0.7)(convm)


    t4 = tf.reshape(convm, shape=[-1, 3, 6, 64])
    deconv4 = tf.image.resize(t4,conv4.shape[1:3],method=resize_method)   
    uconv4 = concatenate([deconv4, pool3])
    uconv4 = PeriodicConv2D(filters=start_filters* 3, kernel_size=kernels)(uconv4)
    uconv4 = LeakyReLU()(uconv4)

    deconv3 = tf.image.resize(uconv4,conv3.shape[1:3],method=resize_method) 
    uconv3 = concatenate([deconv3, pool2])
    uconv3 = PeriodicConv2D(filters=start_filters * 2, kernel_size=kernels)(uconv3)
    uconv3 = LeakyReLU()(uconv3)
    
    deconv2 = tf.image.resize(uconv3,conv2.shape[1:3],method=resize_method)   
    uconv2 = concatenate([deconv2, pool1])
    uconv2 = PeriodicConv2D(filters=start_filters * 1, kernel_size=kernels)(uconv2)
    uconv2 = LeakyReLU()(uconv2)
    
    deconv1 = tf.image.resize(uconv2,conv1.shape[1:3],method=resize_method)   
    uconv1 = concatenate([deconv1, x])
    output_layer= PeriodicConv2D(filters=1, kernel_size=kernels)(uconv1)
    output = LeakyReLU()(output_layer)

    return keras.models.Model(input, output)
    

In [ ]:
def build_unet_small(input_shape, kernels,start_filters):
    x = input = Input(shape=input_shape)
    conv1 = PeriodicConv2D(filters=start_filters* 1, kernel_size=kernels)(x)
    conv1 = LeakyReLU()(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(0.6)(pool1)

    conv2 = PeriodicConv2D(filters=start_filters* 2, kernel_size=kernels)(pool1)
    conv2 = LeakyReLU()(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(0.6)(pool2)
    
    convm = Flatten()(pool2)
    convm = Dense(256,activation="relu")(convm)
    convm = Dropout(0.5)(convm)
    convm = Dense(1*2*64,activation="relu")(convm)
    convm = Dropout(0.5)(convm)


    t4 = tf.reshape(convm, shape=[-1, 1, 2, 64])
    
    deconv2 = tf.image.resize(t4,conv2.shape[1:3],method=resize_method)   
    uconv2 = concatenate([deconv2, pool1])
    uconv2 = PeriodicConv2D(filters=start_filters * 1, kernel_size=kernels)(uconv2)
    uconv2 = LeakyReLU()(uconv2)
    
    deconv1 = tf.image.resize(uconv2,conv1.shape[1:3],method=resize_method)   
    uconv1 = concatenate([deconv1, x])
    output_layer= PeriodicConv2D(filters=1, kernel_size=kernels)(uconv1)
    output = LeakyReLU()(output_layer)

    return keras.models.Model(input, output)
    

In [ ]:
#cnn = build_cnn([64, 64, 64, 64, 1], [5, 5, 5, 5, 5], (48, 96, 2),dr=0.1)
cnn1 = build_cnn([64, 64, 64, 64, 1], [5, 5, 5, 5, 5], (nlat, nlon, nchannels),dr=0.1)
cnn1x2 = build_cnn([64, 64, 64, 64, 1], [5, 5, 5, 5, 5], (nlat, nlon, nchannels),dr=0.1)

cnn11 = build_cnn([64, 64, 64, 64, 1], [5, 5, 5, 5, 5], (nlat, nlon, nchannels),dr=0.1)
cnn11_pool = build_cnn_pool([64, 64, 64, 64, 1], [5, 5, 5, 5, 5], (nlat, nlon, nchannels),dr=0.4)

cnn11xxx = build_cnn([32, 64, 1], [5, 5, 5], (nlat, nlon, nchannels),dr=0.4)

cnn11kiri= build_cnn([128, 8, 1], [5, 5, 5], (nlat, nlon, nchannels),dr=0.3)# for t2m_DJF dr=0.1, for the rest dr=0.3


cnn44 = build_cnn([32, 64, 128, 256, 1], [5, 5, 5, 5, 5], (nlat, nlon, nchannels),dr=0.3)# for t2m_DJF dr=0.2, for the rest dr=0.3
cnn44_pool = build_cnn_pool([32, 64, 128, 256, 1], [5, 5, 5, 5, 1], (nlat, nlon, 2),dr=0.4)

cnn44_acc = build_cnn([32, 64, 128, 256, 1], [5, 5, 5, 5, 5], (nlat, nlon, nchannels),dr=0.4)



cnn5 = build_cnn([64, 64, 64, 64, 1], [3, 3, 3, 3, 3], (nlat, nlon, nchannels),dr=0.1)
cnn55 = build_cnn([64, 64, 64, 64, 1], [3, 3, 3, 3, 3], (nlat, nlon, nchannels),dr=0.4)
#cnn66 = build_cnn([64, 64, 64, 64, 1], [5, 5, 5, 5, 5], (nlat, nlon, nchannels),dr=0.4)

cnn7 = build_cnn([64, 64, 1], [5, 5, 5], (nlat, nlon, nchannels),dr=0.3)# for t2m_DJF dr=0.1, for the rest dr=0.3
cnn8 = build_cnn([64, 64, 1], [3, 3, 3], (nlat, nlon, nchannels),dr=0.3)# for t2m_DJF dr=0.1, for the rest dr=0.3

unet = build_unet(input_shape=(nlat, nlon, nchannels),kernels=3, start_filters=16)
unet1 = build_unet(input_shape=(nlat, nlon, nchannels),kernels=5, start_filters=16)
unet2 = build_unet(input_shape=(nlat, nlon, nchannels),kernels=3, start_filters=32)


unet2_small = build_unet_small(input_shape=(nlat, nlon, nchannels),kernels=3, start_filters=32)
unet3_small = build_unet_small(input_shape=(nlat, nlon, nchannels),kernels=5, start_filters=32)
unet4_small = build_unet_small(input_shape=(nlat, nlon, nchannels),kernels=5, start_filters=16)

unet_small = build_unet_small(input_shape=(nlat, nlon, nchannels),kernels=3, start_filters=16)


In [ ]:
X_hist_coda_19th_ctl_rcp26_shuffled4RNN=X_hist_coda_19th_ctl_rcp26_shuffled.reshape((N_train,nlat,nlon))
y_hist_coda_19th_ctl_rcp26_shuffled4RNN=y_hist_coda_19th_ctl_rcp26_shuffled.reshape((N_train,nlat,nlon))
sst_stacked_coda_20th4RNN=sst_stacked_coda_20th.reshape((N_test,nlat,nlon))

y_test_4RNN=target_coda_20th.reshape((N_test,nlat*nlon))
structure_4RNN= np.arange(y_test_4RNN.shape[1], dtype=float)
structure_4RNN.shape

RNN1=Sequential()
RNN1.add(LSTM(128,input_shape=(nlat,nlon),dropout=0.7,activation="tanh",unroll=True)) # for t2m_DJF dr=0.4, for the rest dr=0.7
RNN1.add(Dense(nlat*nlon,activation="linear"))
RNN1.add(Reshape((nlat,nlon)))
RNN1.summary()

In [ ]:
cnn1.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])
cnn1x2.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])


RNN1.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])


cnn11.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])
cnn11xxx.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])

cnn11kiri.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])


cnn11_pool.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])

cnn44.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])
cnn44_acc.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["accuracy"])

cnn44_pool.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])

cnn5.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])
cnn55.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])
cnn7.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])
cnn8.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])
#cnn66.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])

In [ ]:
unet.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])
unet1.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])
unet2.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])
unet_small.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])
unet2_small.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])
unet3_small.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])
unet4_small.compile(keras.optimizers.Adam(1e-4), loss='mse',metrics=["mae"])


### define checkpoints

In [ ]:

ess = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

ess_rnn = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)


In [ ]:

mc_CNN1_check=readin_data+'best_model_mpi_nov_sst_'+target+'_CNN1_m'+stat_model_member
mc_RNN1_check=readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_RNN1_m'+stat_model_member
mc_auto3_check=readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_auto3_m'+stat_model_member
mc_auto4_check=readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_auto4_m'+stat_model_member
mc_auto5_check=readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_auto5_m'+stat_model_member
mc_auto1_check=readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_auto1_m'+stat_model_member
mc_auto2_check=readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_auto2_m'+stat_model_member
mc_CNN11_check=readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_CNN11_m'+stat_model_member
mc_CNN11kiri_check=readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_CNN11_kiri_m'+stat_model_member
mc_modelC_check=readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_modelC_m'+stat_model_member
mc_CNN44_check=readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_CNN44_m'+stat_model_member
mc_CNN7_check=readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_CNN7_m'+stat_model_member
mc_CNN8_check=readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_CNN8_m'+stat_model_member
mc_unet_check=readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_unet_m'+stat_model_member
mc_unet1_check=readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_unet1_m'+stat_model_member
mc_unet2_check=readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_unet2_m'+stat_model_member

mc_RNN1 = ModelCheckpoint(readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_RNN2_m'+stat_model_member+'.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)


mc_CNN11 = ModelCheckpoint(readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_CNN11_m'+stat_model_member+'.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
mc_CNN11kiri = ModelCheckpoint(readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_CNN11kiri_m'+stat_model_member+'.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)

mc_CNN44 = ModelCheckpoint(readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_CNN44_m'+stat_model_member+'.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
mc_CNN7 = ModelCheckpoint(readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_CNN7_m'+stat_model_member+'.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
mc_CNN8 = ModelCheckpoint(readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_CNN8_m'+stat_model_member+'.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
cmc_unet1 = ModelCheckpoint(readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_unet1_m'+stat_model_member+'.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
mc_unet2 = ModelCheckpoint(readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_unet2_m'+stat_model_member+'.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
mc_unet = ModelCheckpoint(readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_unet_m'+stat_model_member+'.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)

mc_unet_small = ModelCheckpoint(readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_unet1_small_m'+stat_model_member+'.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
mc_unet2_small = ModelCheckpoint(readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_unet2_small_m'+stat_model_member+'.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
mc_unet3_small = ModelCheckpoint(readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_unet3_small_m'+stat_model_member+'.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
mc_unet4_small = ModelCheckpoint(readin_data+'best_modelanomnorm_mpicodaearlyctlrcp26_nov_sst_'+target+'_unet4_small_m'+stat_model_member+'.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)


### Model training

In [ ]:

history_RNN1=RNN1.fit(X_hist_coda_19th_ctl_rcp26_shuffled4RNN,y_hist_coda_19th_ctl_rcp26_shuffled4RNN,epochs=3000,batch_size=128,validation_split=0.2, callbacks=[ess_rnn, mc_RNN1])

val_loss_plot(history=history_RNN1,folder=save_folder,name=target+'_RNN2_m'+stat_model_member)

In [ ]:

history_unet_small=unet_small.fit(X_hist_coda_19th_ctl_rcp26_shuffled,y_hist_coda_19th_ctl_rcp26_shuffled,epochs=3000,batch_size=128,validation_split=0.2, callbacks=[ess, mc_unet_small])

val_loss_plot(history=history_unet_small,folder=save_folder,name=target+'_unet_small_m'+stat_model_member)

In [ ]:

history_unet2_small=unet2_small.fit(X_hist_coda_19th_ctl_rcp26_shuffled,y_hist_coda_19th_ctl_rcp26_shuffled,epochs=3000,batch_size=128,validation_split=0.2, callbacks=[ess, mc_unet2_small])

val_loss_plot(history=history_unet2_small,folder=save_folder,name=target+'_unet2_small_m'+stat_model_member)

In [ ]:

history_unet3_small=unet3_small.fit(X_hist_coda_19th_ctl_rcp26_shuffled,y_hist_coda_19th_ctl_rcp26_shuffled,epochs=3000,batch_size=128,validation_split=0.2, callbacks=[ess, mc_unet3_small])

val_loss_plot(history=history_unet3_small,folder=save_folder,name=target+'_unet3_small_m'+stat_model_member)

In [ ]:

history_unet4_small=unet4_small.fit(X_hist_coda_19th_ctl_rcp26_shuffled,y_hist_coda_19th_ctl_rcp26_shuffled,epochs=3000,batch_size=128,validation_split=0.2, callbacks=[ess, mc_unet4_small])

val_loss_plot(history=history_unet4_small,folder=save_folder,name=target+'_unet4_small_m'+stat_model_member)

In [ ]:
history_cnn44=cnn44.fit(X_hist_coda_19th_ctl_rcp26_shuffled,y_hist_coda_19th_ctl_rcp26_shuffled,epochs=3000,batch_size=128,validation_split=0.2, callbacks=[ess, mc_cnn44])
val_loss_plot(history=history_cnn44,folder=save_folder,name=target+'_cnn44_m'+stat_model_member)

In [ ]:
history_cnn11kiri=cnn11kiri.fit(X_hist_coda_19th_ctl_rcp26_shuffled,y_hist_coda_19th_ctl_rcp26_shuffled,epochs=3000,batch_size=128,validation_split=0.2, callbacks=[ess, mc_cnn11kiri])
val_loss_plot(history=history_cnn11kiri,folder=save_folder,name=target+'_cnn11kiri_m'+stat_model_member)

In [ ]:
history_cnn8=cnn8.fit(X_hist_coda_19th_ctl_rcp26_shuffled,y_hist_coda_19th_ctl_rcp26_shuffled,epochs=3000,batch_size=128,validation_split=0.2, callbacks=[ess, mc_cnn8])
val_loss_plot(history=history_cnn8,folder=save_folder,name=target+'_cnn8_m'+stat_model_member)

In [ ]:
history_cnn7=cnn7.fit(X_hist_coda_19th_ctl_rcp26_shuffled,y_hist_coda_19th_ctl_rcp26_shuffled,epochs=3000,batch_size=128,validation_split=0.2, callbacks=[ess, mc_cnn7])
val_loss_plot(history=history_cnn7,folder=save_folder,name=target+'_cnn7_m'+stat_model_member)

In [ ]:

history_unet2=unet2.fit(X_hist_coda_19th_ctl_rcp26_shuffled,y_hist_coda_19th_ctl_rcp26_shuffled,epochs=3000,batch_size=128,validation_split=0.2, callbacks=[ess, mc_unet2])

val_loss_plot(history=history_unet2,folder=save_folder,name=target+'_unet2_m'+stat_model_member)

In [ ]:

history_unet1=unet1.fit(X_hist_coda_19th_ctl_rcp26_shuffled,y_hist_coda_19th_ctl_rcp26_shuffled,epochs=3000,batch_size=128,validation_split=0.2, callbacks=[ess, mc_unet1])

val_loss_plot(history=history_unet1,folder=save_folder,name=target+'_unet1_m'+stat_model_member)

In [ ]:

history_unet=unet.fit(X_hist_coda_19th_ctl_rcp26_shuffled,y_hist_coda_19th_ctl_rcp26_shuffled,epochs=3000,batch_size=128,validation_split=0.2, callbacks=[ess, mc_unet])

val_loss_plot(history=history_unet,folder=save_folder,name=target+'_unet_m'+stat_model_member)

### check over under fitting

In [ ]:
val_loss_list=[]
loss_list=[]

In [ ]:
loss =history_cnn44.history["loss"]
val_loss =history_cnn44.history["val_loss"]
epochs=range(1,len(loss)+1)

print(np.min(val_loss))
val_loss_list.append(np.min(val_loss))

print(np.min(loss))
loss_list.append(np.min(loss))
plt.figure()

plt.plot(epochs,loss,"b",color="blue",label="Training loss")
plt.plot(epochs,val_loss,"b",color="red",label="Validation loss")
plt.title("Loss Curves for cnn44")
plt.legend()
plt.show()

In [ ]:
loss =history_RNN1.history["loss"]
val_loss =history_RNN1.history["val_loss"]
epochs=range(1,len(loss)+1)

print(np.min(val_loss))
val_loss_list.append(np.min(val_loss))

print(np.min(loss))
loss_list.append(np.min(loss))
plt.figure()

plt.plot(epochs,loss,"b",color="blue",label="Training loss")
plt.plot(epochs,val_loss,"b",color="red",label="Validation loss")
plt.title("Loss Curves for RNN1")
plt.legend()
plt.show()

In [ ]:
loss =history_cnn8.history["loss"]
val_loss =history_cnn8.history["val_loss"]
epochs=range(1,len(loss)+1)

print(np.min(val_loss))
val_loss_list.append(np.min(val_loss))

print(np.min(loss))
loss_list.append(np.min(loss))
plt.figure()

plt.plot(epochs,loss,"b",color="blue",label="Training loss")
plt.plot(epochs,val_loss,"b",color="red",label="Validation loss")
plt.title("Loss Curves for cnn8")
plt.legend()
plt.show()

In [ ]:
loss =history_cnn7.history["loss"]
val_loss =history_cnn7.history["val_loss"]
epochs=range(1,len(loss)+1)

print(np.min(val_loss))
val_loss_list.append(np.min(val_loss))

print(np.min(loss))
loss_list.append(np.min(loss))
plt.figure()

plt.plot(epochs,loss,"b",color="blue",label="Training loss")
plt.plot(epochs,val_loss,"b",color="red",label="Validation loss")
plt.title("Loss Curves for cnn7")
plt.legend()
plt.show()

In [ ]:
loss =history_cnn11kiri.history["loss"]
val_loss =history_cnn11kiri.history["val_loss"]
epochs=range(1,len(loss)+1)

print(np.min(val_loss))
val_loss_list.append(np.min(val_loss))

print(np.min(loss))
loss_list.append(np.min(loss))
plt.figure()

plt.plot(epochs,loss,"b",color="blue",label="Training loss")
plt.plot(epochs,val_loss,"b",color="red",label="Validation loss")
plt.title("Loss Curves for cnn11kiri")
plt.legend()
plt.show()

In [ ]:
loss =history_unet1.history["loss"]
val_loss =history_unet1.history["val_loss"]
epochs=range(1,len(loss)+1)

print(np.min(val_loss))
val_loss_list.append(np.min(val_loss))

print(np.min(loss))
loss_list.append(np.min(loss))
plt.figure()

plt.plot(epochs,loss,"b",color="blue",label="Training loss")
plt.plot(epochs,val_loss,"b",color="red",label="Validation loss")
plt.title("Loss Curves for unet1")
plt.legend()
plt.show()

In [ ]:
loss =history_unet2.history["loss"]
val_loss =history_unet2.history["val_loss"]
epochs=range(1,len(loss)+1)

print(np.min(val_loss))
val_loss_list.append(np.min(val_loss))

print(np.min(loss))
loss_list.append(np.min(loss))
plt.figure()

plt.plot(epochs,loss,"b",color="blue",label="Training loss")
plt.plot(epochs,val_loss,"b",color="red",label="Validation loss")
plt.title("Loss Curves for unet2")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8, 4), dpi= 200)
loss =history_unet.history["loss"]
val_loss =history_unet.history["val_loss"]
epochs=range(1,len(loss)+1)

print(np.min(val_loss))
val_loss_list.append(np.min(val_loss))

print(np.min(loss))
loss_list.append(np.min(loss))
plt.figure()

plt.plot(epochs,loss,"b",color="blue",label="Training loss")
plt.plot(epochs,val_loss,"b",color="red",label="Validation loss")
plt.title("Loss Curves for unet")

plt.title("VL: "+str(round(np.min(val_loss),2)),loc="left")
plt.title("TL: "+str(round(np.min(loss),2)),loc="right")
plt.legend()
plt.show()

In [ ]:
loss =history_unet3_small.history["loss"]
val_loss =history_unet3_small.history["val_loss"]
epochs=range(1,len(loss)+1)

print(np.min(val_loss))
val_loss_list.append(np.min(val_loss))

print(np.min(loss))
loss_list.append(np.min(loss))
plt.figure()

plt.plot(epochs,loss,"b",color="blue",label="Training loss")
plt.plot(epochs,val_loss,"b",color="red",label="Validation loss")
plt.title("Loss Curves for unet3_small")
plt.legend()
plt.show()

In [ ]:
loss =history_unet2_small.history["loss"]
val_loss =history_unet2_small.history["val_loss"]
epochs=range(1,len(loss)+1)

print(np.min(val_loss))
val_loss_list.append(np.min(val_loss))

print(np.min(loss))
loss_list.append(np.min(loss))
plt.figure()

plt.plot(epochs,loss,"b",color="blue",label="Training loss")
plt.plot(epochs,val_loss,"b",color="red",label="Validation loss")
plt.title("Loss Curves for unet2_small")
plt.legend()
plt.show()

In [ ]:
loss =history_unet4_small.history["loss"]
val_loss =history_unet4_small.history["val_loss"]
epochs=range(1,len(loss)+1)

print(np.min(val_loss))
val_loss_list.append(np.min(val_loss))

print(np.min(loss))
loss_list.append(np.min(loss))
plt.figure()

plt.plot(epochs,loss,"b",color="blue",label="Training loss")
plt.plot(epochs,val_loss,"b",color="red",label="Validation loss")
plt.title("Loss Curves for unet4_small")
plt.legend()
plt.show()

In [ ]:
loss =history_unet_small.history["loss"]
val_loss =history_unet_small.history["val_loss"]
epochs=range(1,len(loss)+1)

print(np.min(val_loss))
val_loss_list.append(np.min(val_loss))

print(np.min(loss))
loss_list.append(np.min(loss))
plt.figure()

plt.plot(epochs,loss,"b",color="blue",label="Training loss")
plt.plot(epochs,val_loss,"b",color="red",label="Validation loss")
plt.title("Loss Curves for unet_small")
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(11, 7))
plt.plot(val_loss_list,"bo",color="red",label="Min. Validation loss")
plt.title("Minimum Loss")
plt.plot(loss_list,"bo",color="blue",label="Min. Training loss")
plt.legend()
plt.xticks([0, 1, 2,3,4,5,6,7,8,9,10,11], ["cnn44","RNN1","cnn8","cnn7","cnn11kiri","unet1","unet2","unet","unet3_small","unet2_small","unet4_small","unet_small"],rotation=90)


In [ ]:
plt.show()
fig.savefig(save_folder+"min_val_loss_"+stat_model_member+"_"+target+".pdf")
fig.savefig(save_folder+"min_val_loss_"+stat_model_member+"_"+target+".png")